In [ ]:
# 1-1. 필수 라이브러리 설치 (Colab 셀에서 실행)
!pip install sentence-transformers pandas tqdm

# 1-2. 파일 업로드
# 좌측 폴더 아이콘을 클릭하여 'java_dpo.json', 'python_dpo.json', 'java_mega_vuln.json' 파일을 업로드합니다.

import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os
import numpy as np
import ast # 벡터 저장을 위해 필요

# --- 설정 변수 ---
MODEL_NAME = "sentence-transformers/multi-qa-distilbert-cos-v1"
OUTPUT_FILENAME = "secure_coding_embeddings.csv"

FILE_PATHS = [
    "java_dpo.json",
    "python_dpo.json",
    "java_mega_vuln.json"
]

# --- 1. 데이터 로드 및 RAG 청크 생성 (필드 매핑 로직 추가) ---
def load_and_preprocess_data(file_paths: list) -> pd.DataFrame:
    """JSON 파일을 로드하고 필드명을 통합하여 RAG 청크를 생성합니다."""
    data = []
    existing_files = [f for f in file_paths if os.path.exists(f)]

    if not existing_files:
        print("🚨 오류: 파일을 찾을 수 없습니다. Colab에 JSON 파일을 업로드했는지 확인하세요.")
        return pd.DataFrame()

    print(f"✅ 다음 파일들을 로드합니다: {existing_files}")

    for file_path in existing_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            file_data = json.load(f)
            data.extend(file_data)

    processed_data = []
    for i, item in enumerate(data):

        # --- 핵심: 필드 통합 매핑 ---
        # 1. 취약점 설명 (vulnerability_description 또는 vulnerability_summary 사용)
        description = item.get('vulnerability_description', item.get('vulnerability_summary', 'N/A'))
        # 2. 안전한 코드 (safe_code 또는 patched_code 사용)
        safe_code = item.get('safe_code', item.get('patched_code', 'N/A'))
        # 3. 취약한 코드
        vulnerable_code = item.get('vulnerable_code', 'N/A')

        # CodeBERT 임베딩에 최적화된 결합 텍스트 (Code와 취약점 Context 결합)
        context = (
            f"CWE-ID: {item.get('cwe_id', 'N/A')}\n"
            f"취약점 설명: {description}\n"
            f"취약한 코드:\n{vulnerable_code}\n"
            f"안전한 코드:\n{safe_code}"
        )

        # Qdrant Payload로 사용할 메타데이터 (JSON 직렬화를 위해 준비)
        payload = json.dumps({
            "cwe_id": item.get('cwe_id', 'N/A'),
            "description": description,
            "safe_code": safe_code,
            "vulnerable_code": vulnerable_code,
            # 추가된 필드도 포함 가능
            "abstract_vulnerable_code": item.get('abstract_vulnerable_code', 'N/A'),
            "source_file": file_path
        })

        processed_data.append({
            "id": i, # Qdrant Point ID로 사용할 고유 ID
            "text": context,
            "payload_json": payload
        })

    return pd.DataFrame(processed_data)

# --- 2. 임베딩 생성 및 파일 저장 ---
def generate_and_save_embeddings(df: pd.DataFrame, model_name: str, output_path: str):
    """데이터를 임베딩하고 벡터와 메타데이터를 CSV 파일로 저장합니다."""

    if df.empty:
        print("데이터프레임이 비어 있어 임베딩을 진행할 수 없습니다.")
        return

    print(f"Loading embedding model: {model_name}...")
    model = SentenceTransformer(model_name)
    vector_dim = model.get_sentence_embedding_dimension()
    print(f"Model loaded. Vector dimension: {vector_dim}")

    print(f"Generating embeddings for {len(df)} data points...")
    vectors = model.encode(
        df['text'].tolist(),
        show_progress_bar=True,
        normalize_embeddings=True,
        batch_size=32
    )

    # 벡터를 문자열로 변환하여 CSV에 저장
    df['vector'] = [vec.tolist() for vec in vectors]

    # 최종 CSV 저장: ID, 벡터, 메타데이터 JSON
    final_df = df[['id', 'vector', 'payload_json']]
    final_df.to_csv(output_path, index=False)

    print(f"\n✅ 임베딩 생성 및 파일 저장 완료: '{output_path}'")
    print("Colab 좌측 메뉴에서 이 파일을 다운로드하여 2단계에서 사용하세요.")
    print(f"총 데이터 포인트: {len(df)}, 벡터 차원: {vector_dim}")

# 실행
df = load_and_preprocess_data(FILE_PATHS)
generate_and_save_embeddings(df, MODEL_NAME, OUTPUT_FILENAME)#%%
# 1-1. 필수 라이브러리 설치 (Colab 셀에서 실행)
!pip install sentence-transformers pandas tqdm

# 1-2. 파일 업로드
# 좌측 폴더 아이콘을 클릭하여 'java_dpo.json', 'python_dpo.json', 'java_mega_vuln.json' 파일을 업로드합니다.

import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os
import numpy as np
import ast # 벡터 저장을 위해 필요

# --- 설정 변수 ---
MODEL_NAME = "sentence-transformers/multi-qa-distilbert-cos-v1"
OUTPUT_FILENAME = "secure_coding_embeddings.csv"

FILE_PATHS = [
    "java_dpo.json",
    "python_dpo.json",
    "java_mega_vuln.json"
]

# --- 1. 데이터 로드 및 RAG 청크 생성 (필드 매핑 로직 추가) ---
def load_and_preprocess_data(file_paths: list) -> pd.DataFrame:
    """JSON 파일을 로드하고 필드명을 통합하여 RAG 청크를 생성합니다."""
    data = []
    existing_files = [f for f in file_paths if os.path.exists(f)]

    if not existing_files:
        print("🚨 오류: 파일을 찾을 수 없습니다. Colab에 JSON 파일을 업로드했는지 확인하세요.")
        return pd.DataFrame()

    print(f"✅ 다음 파일들을 로드합니다: {existing_files}")

    for file_path in existing_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            file_data = json.load(f)
            data.extend(file_data)

    processed_data = []
    for i, item in enumerate(data):

        # --- 핵심: 필드 통합 매핑 ---
        # 1. 취약점 설명 (vulnerability_description 또는 vulnerability_summary 사용)
        description = item.get('vulnerability_description', item.get('vulnerability_summary', 'N/A'))
        # 2. 안전한 코드 (safe_code 또는 patched_code 사용)
        safe_code = item.get('safe_code', item.get('patched_code', 'N/A'))
        # 3. 취약한 코드
        vulnerable_code = item.get('vulnerable_code', 'N/A')

        # CodeBERT 임베딩에 최적화된 결합 텍스트 (Code와 취약점 Context 결합)
        context = (
            f"CWE-ID: {item.get('cwe_id', 'N/A')}\n"
            f"취약점 설명: {description}\n"
            f"취약한 코드:\n{vulnerable_code}\n"
            f"안전한 코드:\n{safe_code}"
        )

        # Qdrant Payload로 사용할 메타데이터 (JSON 직렬화를 위해 준비)
        payload = json.dumps({
            "cwe_id": item.get('cwe_id', 'N/A'),
            "description": description,
            "safe_code": safe_code,
            "vulnerable_code": vulnerable_code,
            # 추가된 필드도 포함 가능
            "abstract_vulnerable_code": item.get('abstract_vulnerable_code', 'N/A'),
            "source_file": file_path
        })

        processed_data.append({
            "id": i, # Qdrant Point ID로 사용할 고유 ID
            "text": context,
            "payload_json": payload
        })

    return pd.DataFrame(processed_data)

# --- 2. 임베딩 생성 및 파일 저장 ---
def generate_and_save_embeddings(df: pd.DataFrame, model_name: str, output_path: str):
    """데이터를 임베딩하고 벡터와 메타데이터를 CSV 파일로 저장합니다."""

    if df.empty:
        print("데이터프레임이 비어 있어 임베딩을 진행할 수 없습니다.")
        return

    print(f"Loading embedding model: {model_name}...")
    model = SentenceTransformer(model_name)
    vector_dim = model.get_sentence_embedding_dimension()
    print(f"Model loaded. Vector dimension: {vector_dim}")

    print(f"Generating embeddings for {len(df)} data points...")
    vectors = model.encode(
        df['text'].tolist(),
        show_progress_bar=True,
        normalize_embeddings=True,
        batch_size=32
    )

    # 벡터를 문자열로 변환하여 CSV에 저장
    df['vector'] = [vec.tolist() for vec in vectors]

    # 최종 CSV 저장: ID, 벡터, 메타데이터 JSON
    final_df = df[['id', 'vector', 'payload_json']]
    final_df.to_csv(output_path, index=False)

    print(f"\n✅ 임베딩 생성 및 파일 저장 완료: '{output_path}'")
    print("Colab 좌측 메뉴에서 이 파일을 다운로드하여 2단계에서 사용하세요.")
    print(f"총 데이터 포인트: {len(df)}, 벡터 차원: {vector_dim}")

# 실행
df = load_and_preprocess_data(FILE_PATHS)
generate_and_save_embeddings(df, MODEL_NAME, OUTPUT_FILENAME)